# Tidegate Desktop Tool Demo

## Basic import statements

In [ ]:
import arcpy
import numpy

import tidegates  # <-- our module

## Inputs that come from the Toolbox

In [ ]:
workspace = r'F:\phobson\Tidegates\MB_InputLayers.gdb'
dem = r'F:\phobson\Tidegates\MB_InputLayers.gdb\cape_dem'
polygons = r'F:\phobson\Tidegates\MB_InputLayers.gdb\ZOI'
tidegate_column = 'GeoID'
sea_level_rise = 1
storm_surge = '10-yr'
filename = 'flood_10yr_01ft'


def make_name(surge, slr):
    return "PF_S{:s}_{:02d}ft".format(surge.replace('-',''), slr)

## Create a new scenario

In [ ]:
SEALEVELRISE = numpy.arange(0, 7) # 0 through 6 in 1-ft increments
for surge in tidegates.SURGES.keys():
    for slr in SEALEVELRISE:
        layer_name = make_name(surge, slr)
        print('\t' + layer_name)
        with tidegates.utils.WorkSpace(workspace), tidegates.utils.OverwriteState(True):
            flood = tidegates.flood_area(
                dem=dem, 
                polygons=polygons, 
                tidegate_column=tidegate_column, 
                sea_level_rise=slr, 
                storm_surge=surge, 
                filename=layer_name
            )
            
            arcpy.management.AddField(
                in_table=flood.getOutput(0),
                field_name='SLR',
                field_type='DOUBLE'
            )
            
            arcpy.management.AddField(
                in_table=flood.getOutput(0),
                field_name='surge',
                field_type='TEXT',
                field_length=10
            )
            
            cur = arcpy.UpdateCursor(flood.getOutput(0))
            for row in cur:
                row.SLR = slr
                row.surge = surge
                cur.updateRow(row)

            del cur, row

In [ ]:
layers = []
for slr in range(7):
    for surge in tidegates.SURGES.keys():
        layer_name = make_name(surge, slr)
        layers.append(layer_name)

with tidegates.utils.WorkSpace(workspace), tidegates.utils.OverwriteState(True):
    arcpy.management.Merge(layers, 'merged_01')

## Assess the impacts of the scenario

In [ ]:
with tidegates.utils.WorkSpace(workspace):
    impacts = tidegates.assess_impact(
        flood_layer='merged_01',
        input_gdb=workspace,    
        overwrite=True
    )